# Notebook to test the reading of an AOR

In [2]:
from xml.etree.ElementTree import ElementTree as ET

aorfile = '../test/70_0808.aor'
tree = ET(file=aorfile)
vector = tree.find('list/vector')
# Extract Target and Instrument from each AOR
targets = [item.text for item in vector.findall('Request/target/name')]
instruments = [item.text for item in vector.findall('Request/instrument/data/InstrumentName')]
targets

['sgra', 'm83-1', 'The Brig-1', 'SgrC-1', 'Dust Ridge-1', 'm83-1']

In [3]:
# Abstract
abstract = tree.find('list/ProposalInfo/ProposalAbstract')
print(abstract.text)

FIFI-LS GTO Cycle 8


In [9]:
import numpy as np
instruments = np.array(instruments)
idx = instruments == 'FIFI-LS'

In [10]:
idx

array([ True,  True,  True,  True,  True,  True, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True])

# Extract images from pdf

The PyPDF2 does not work.

The package pymupdf works.
It has to be installed with pip (pip install pymupdf).

In [7]:
import PyPDF2
from PIL import Image
    

In [19]:
pdf = PyPDF2.PdfFileReader(open("../test/08_0226_attachment.pdf", "rb"))
print(f"The total number of pages in the pdf document is {pdf.numPages}")
print(f'PDF Metadata is {pdf.documentInfo}')
print(pdf.getDocumentInfo())
#print(f'PDF File Author is {input1.documentInfo["/Author"]}')
#print(f'PDF File Creator is {input1.documentInfo["/Creator"]}')
#pdf.getPage(0)['/Resources']['/Properties']['/MC0']['/MYOBJECT'].getData()

# Objects
for obj in pdf.resolvedObjects:
    print(obj)

#print(pdf.resolvedObjects)

The total number of pages in the pdf document is 6
PDF Metadata is {'/Producer': 'iText® 5.5.10 ©2000-2015 iText Group NV (AGPL-version)', '/CreationDate': "D:20210730162927-07'00'", '/ModDate': "D:20210730162927-07'00'"}
{'/Producer': 'iText® 5.5.10 ©2000-2015 iText Group NV (AGPL-version)', '/CreationDate': "D:20210730162927-07'00'", '/ModDate': "D:20210730162927-07'00'"}
(0, 97)
(0, 3)
(0, 4)
(0, 7)
(0, 10)
(0, 13)
(0, 16)
(0, 19)
(0, 98)


In [7]:
print (input1.resolvedObjects)
pages = list(input1.pages)
print('pages' , pages)
for ipage in range(input1.numPages):
    page0 = input1.getPage(ipage)
    print('text: ', page0.extractText())
    #xObject = page0['/Type']['/XObject'].getObject()
    print('type: ',page0['/Type'])
    if '/XObject' in page0['/Type']:
        xObject = page0['/Type']['/XObject'].getObject()
        print('resources found')
    else:
        print('no resources found')


    for obj in xObject:
        print('obj',xObject[obj]['/Subtype'] )
        if xObject[obj]['/Subtype'] == '/Image':
            print('image')
            size = (xObject[obj]['/Width'], xObject[obj]['/Height'])
            data = xObject[obj].getData()
            if xObject[obj]['/ColorSpace'] == '/DeviceRGB':
                mode = "RGB"
            else:
                mode = "P"

            if '/Filter' in xObject[obj]:
                if xObject[obj]['/Filter'] == '/FlateDecode':
                    img = Image.frombytes(mode, size, data)
                    img.save('../test/'+obj[1:] + ".png")
                elif xObject[obj]['/Filter'] == '/DCTDecode':
                    img = open('../test/'+obj[1:] + ".jpg", "wb")
                    img.write(data)
                    img.close()
                elif xObject[obj]['/Filter'] == '/JPXDecode':
                    img = open('../test/'+obj[1:] + ".jp2", "wb")
                    img.write(data)
                    img.close()
                elif xObject[obj]['/Filter'] == '/CCITTFaxDecode':
                    img = open(obj[1:] + ".tiff", "wb")
                    img.write(data)
                    img.close()
            else:
                img = Image.frombytes(mode, size, data)
                img.save(obj[1:] + ".png")

The total number of pages in the pdf document is 6
PDF Metadata is {'/Producer': 'iText® 5.5.10 ©2000-2015 iText Group NV (AGPL-version)', '/CreationDate': "D:20210730162927-07'00'", '/ModDate': "D:20210730162927-07'00'"}
{'/Producer': 'iText® 5.5.10 ©2000-2015 iText Group NV (AGPL-version)', '/CreationDate': "D:20210730162927-07'00'", '/ModDate': "D:20210730162927-07'00'"}
{(0, 97): {'/Type': '/Catalog', '/Pages': IndirectObject(3, 0)}, (0, 3): {'/Type': '/Pages', '/Count': 6, '/Kids': [IndirectObject(4, 0), IndirectObject(7, 0), IndirectObject(10, 0), IndirectObject(13, 0), IndirectObject(16, 0), IndirectObject(19, 0)]}, (0, 4): {'/Type': '/Page', '/MediaBox': [0, 0, 612, 792], '/Resources': {'/XObject': {'/Xf1': IndirectObject(1, 0)}}, '/ArtBox': [36, 54, 572, 759], '/Contents': IndirectObject(2, 0), '/Parent': IndirectObject(3, 0)}, (0, 7): {'/Type': '/Page', '/MediaBox': [0, 0, 612, 792], '/Resources': {'/XObject': {'/Xf2': IndirectObject(5, 0)}}, '/ArtBox': [36, 54, 572, 759], '/

NameError: name 'xObject' is not defined

In [62]:
# How to kill Adobe on MAC
import psutil
for p in psutil.process_iter():
    #print(p.name())
    if p.name() == 'AdobeReader':
        p.kill()

In [9]:
import fitz
import os 
aor = '08_0095'
directory = os.path.join("..","test",aor)
if not os.path.exists(directory):
    os.makedirs(directory)
doc = fitz.open(os.path.join("..","test",aor+"_attachment.pdf"))
for i in range(len(doc)):
    for img in doc.getPageImageList(i):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        if pix.n < 5:       # this is GRAY or RGB
            pix.writePNG(os.path.join("..","test",aor,"p%s-%s.png") % (i, xref))
        else:               # CMYK: convert to RGB first
            pix1 = fitz.Pixmap(fitz.csRGB, pix)
            pix1.writePNG(os.path.join("..","test",aor,"p%s-%s.png") % (i, xref))
            pix1 = None
        pix = None

# Flight plan table

In [4]:

import xml.etree.ElementTree as ET
aorfile = '../test/202105_FI_MAXINE_WX36.misxml'
tree = ET.parse(aorfile)
root = tree.getroot()

#root.tag
#root.attrib

for flightplan in root.iter('FlightPlan'):
    flightTime = flightplan.attrib['FltTime']
    flightName = flightplan.attrib['Id']
    departure = flightplan.attrib['DepartureTime']
    arrival = flightplan.attrib['ArrivalTime']
    print(flightName, flightTime, departure, arrival)

table = ''
fmt = '{0:s} & {1:s} & {2:s} & {3:s} & {4:s} & {5:s} & {6:s}\\\\\\n'
for leg in root.iter('Leg'):
    name = leg.find('Name').text
    start = leg.find('Start').text
    duration = leg.find('Duration').text
    altitude = leg.find('Alt').text
    planid = leg.find('ObsPlanID')
    if planid is None:
        planid = ''
    else:
        planid = str(planid.text)
        planid = planid.replace('_','\_')
        if planid is None:
            planid = ''
    elevation = leg.find('Elev')
    if elevation is not None:
        elstart = elevation.attrib['start']
        elend = elevation.attrib['end']
        elevation = elstart+'-'+elend
    else:
        elend = ''
        elstart=''
        elevation = ''
    ID = leg.attrib['id']
    ftype = leg.attrib['type']
    if ftype == 'Absolute':
        name = 'Climb'
    elif ftype == 'Arrival':
        name = 'Arrival'
    planid = planid.replace('None','')
    line = fmt.format(ID, name, planid, elevation, start, duration[0:5], altitude)
    print(line)
    
ID = planid = elevation = duration = ''
name = 'Landing'
start = arrival[11:19]
altitude = '0 ft'
line = fmt.format(ID, name, planid, elevation, start, duration, altitude)
print(line)
name = 'Total flight time'
start = altitude = ''
duration = flightTime
line = fmt.format(ID, name, planid, elevation, start, duration, altitude)
print(line)
    
    
# Save in a latex file - create a main.tex that includes it and run pdflatex

202105_FI_MAXINE 08:26:53 2021-06-05T03:45:00Z 2021-06-05T12:11:52Z
1 & Departure &  &  & 03:45:00 & 00:02 & 10000 ft\\\n
2 & Climb &  &  & 03:48:46 & 00:08 & 35000 ft\\\n
3 & Climb &  &  & 03:58:38 & 00:14 & 36000 ft\\\n
4 & Setup &  & 24.3-25.0 & 04:14:20 & 00:30 & 38000 ft\\\n
5 & m83-1 & 70\_0808 & 28.3-28.4 & 04:44:23 & 00:53 & 39000 ft\\\n
6 & IRAS 12112+0305 & 08\_0095 & 54.4-43.4 & 05:38:31 & 01:17 & 39000/35/41000 ft\\\n
7 & NGC6946_Nuc & 09\_0198 & 32.6-44.5 & 07:00:08 & 01:45 & 41000/75/43000 ft\\\n
8 & Sgr A & 71\_0024 & 30.3-31.1 & 08:51:17 & 00:54 & 43000 ft\\\n
9 & Mrk 0290 & 08\_0226 & 54.8-42.1 & 09:49:34 & 01:35 & 43000 ft\\\n
10 & Arrival &  &  & 11:26:29 & 00:45 & 43000 ft\\\n
 & Landing &  &  & 12:11:52 &  & 0 ft\\\n
 & Total flight time &  &  &  & 08:26:53 & \\\n


### Using the module

In [17]:
from fififly.flightplan import flightTable
aorfile = '../test/202105_FI_MAXINE_WX36.misxml'
table = flightTable(aorfile)
for t in table:
    print(t)


\begin{table}
\begin{center}
\begin{tabular}{rrccrrr}
\hline
\hline
Leg & Target & AOR_ID & Elevation & UTC start & Duration & Altitude\\
\hline
\hline
1  & Departure &  &  & 03:45:00 & 00:02 & 10000 ft\\
2  & Climb &  &  & 03:48:46 & 00:08 & 35000 ft\\
3  & Climb &  &  & 03:58:38 & 00:14 & 36000 ft\\
4  & Setup &  & 24.3-25.0 & 04:14:20 & 00:30 & 38000 ft\\
5  & m83-1 & 70_0808 & 28.3-28.4 & 04:44:23 & 00:53 & 39000 ft\\
6  & IRAS 12112+0305 & 08_0095 & 54.4-43.4 & 05:38:31 & 01:17 & 39000/35/41000 ft\\
7  & NGC6946_Nuc & 09_0198 & 32.6-44.5 & 07:00:08 & 01:45 & 41000/75/43000 ft\\
8  & Sgr A & 71_0024 & 30.3-31.1 & 08:51:17 & 00:54 & 43000 ft\\
9  & Mrk 0290 & 08_0226 & 54.8-42.1 & 09:49:34 & 01:35 & 43000 ft\\
10 & Arrival &  &  & 11:26:29 & 00:45 & 43000 ft\\
   & Landing &   &   & 12:11:52 &   & 0 ft\\
   & Total flight time &   &   &  & 08:26:53 & \\
\hline
\hline
\end{tabular}
\end{center}
\end{table}


In [148]:
legs = tree.find('FlightPlan/Legs')

## Table of legs

In [138]:
legs = [item for item in flightplan.findall('Legs')]

AttributeError: 'NoneType' object has no attribute 'findall'

TypeError: element indices must be integers

# Compile and display

In [16]:
import platform
import subprocess
print(platform.system().lower() )
output = subprocess.run(["ls", "-l", "."], capture_output=True)
subprocess.getoutput('ls ~')

darwin


'09_0038_backup.aor\nA85\nATT\nAdministration\nApplications\nBooks\nCIAO.ipynb\nCertificates\nChandra\nCodes\nCommunityDay\nDesktop\nDocuments\nDownloads\nFIFI-LS\nFORCAST\nHAWC\nHelp\nIDL\nIDLWorkspace\nIDLpipeline\nInterviews\nIrsLow\nLibrary\nModel\nMovies\nMusic\nNotes\nOLDPC\nOldDesktop\nPACS\nPapers\nPictures\nProposals\nPublic\nPython\nReferee\nReviews\nSciPy2019\nSites\nSpitzer\nTalks\nTeleTalk\nUsers\nWeb\nadam\nanaconda3\nbin\ncxcds_param4\ndemo.docx\nhcss\nmopex-script-env.bash\nmopex-script-env.csh\nold.spyder\nold.spyder-py3\nsofia'

In [ ]:
# works with Python3
import os
import platform
import subprocess

# TeX source filename
tex_filename = 'my_doc.tex'
filename, ext = os.path.splitext(tex_filename)
# the corresponding PDF filename
pdf_filename = filename + '.pdf'

# compile TeX file
subprocess.run(['pdflatex', '-interaction=nonstopmode', '-output-directory=pdf' tex_filename])

# check if PDF is successfully generated
if not os.path.exists(pdf_filename):
    raise RuntimeError('PDF output not found')

# open PDF with platform-specific command
if platform.system().lower() == 'darwin':
    subprocess.run(['open', pdf_filename])
elif platform.system().lower() == 'windows':
    os.startfile(pdf_filename)
elif platform.system().lower() == 'linux':
    subprocess.run(['xdg-open', pdf_filename])
else:
    raise RuntimeError('Unknown operating system "{}"'.format(platform.system()))


# Program

In [1]:
from fififly.flightplan import startDescription, writeLatex, runLatex
aorfile = '../test/202105_FI_MAXINE_WX36'
cycle = 'OC8M'

# Create description json file
startDescription(aorfile)

# Create latex file
writeLatex(aorfile, cycle)

# Compile latex file and create pdf file
runLatex(aorfile)

obBlockID OB_70_0808_02
comment  # 122 minutes requested in DCS<br># 120 minutes planned in DCS<br># 54 of 120 minutes in this series<br># 66 on MARIO<br># 54 on MAXINE (this flight)
comment  # 122 minutes requested in DCS<br># 120 minutes planned in DCS<br># 54 of 120 minutes in this series<br># 66 on MARIO<br># 54 on MAXINE (this flight)
aorfile  ../test/70_0808.aor
abstract  FIFI-LS GTO Cycle 8
obBlockID OB_08_0095_07
comment  # 70 minutes requested in DCS<br># 70 minutes planned in DCS<br># 70 of 70 minutes in this series
comment  # 70 minutes requested in DCS<br># 70 minutes planned in DCS<br># 70 of 70 minutes in this series
../test/08_0095/1.png
../test/08_0095/2.png
aorfile  ../test/08_0095.aor
abstract  A comprehensive stud
obBlockID OB_09_0198_01
comment  # 711 minutes requested in DCS<br># 406 minutes planned in DCS<br># 100 of 245 minutes in this series<br># 50 on MADDILYN<br># 50 on MATTHEW<br># 100 on MAXINE (this flight)<br># 45 on MIRIAM
comment  # 711 minutes requested

In [3]:
import json
with open('../test/MAXINE.json', 'r') as jf:
    data = json.load(jf)
    
print(data['ObsBlocks'])

['OB_70_0808_02', 'OB_08_0095_07', 'OB_09_0198_01', 'OB_71_0024_14', 'OB_08_0226_04']


# Retrieve DSS2 image

Using skyview retrieve image from DSS2 (red) and save to a file.
This can be done with coordinates or source name (it's better to use coordinates).
The field width and heigth should be specified.

In [28]:
from astroquery.skyview import SkyView
from astropy import units as u
paths = SkyView.get_images(position='M 82', survey='DSS2 Red', pixels=1000,width=0.25*u.deg, height=0.25*u.deg)
paths[0].writeto('../test/M82.fits', overwrite=True)

In [30]:
# Example (08_0095 - obsBlock 01 - ra=15.582570666666667, dec=23.50314)
paths = SkyView.get_images(position='233.738559, 15.5825706', survey='DSS2 Red', pixels=1000, width=0.25*u.deg, height=0.25*u.deg)
paths[0].writeto('../test/test.fits', overwrite=True)